# Hybrid Local DSPy RAG with Ollama

This notebook implements a French RAG (Retrieval-Augmented Generation) system using:
- Hybrid retrieval (CamemBERT + BM25)
- DSPy for orchestration
- Mistral via Ollama for language generation
- Metal (MPS) acceleration for Mac
- MLX for optimized computation

In [46]:
# 1. Install required libraries
!pip install mlx mlx-torch
!pip install transformers rank_bm25 nltk torch dspy-ai --quiet
!pip install sentencepiece==0.1.99
!pip install ollama
!pip install --upgrade dspy-ai

import nltk
nltk.download('punkt')

ERROR: Could not find a version that satisfies the requirement mlx-torch (from versions: none)
ERROR: No matching distribution found for mlx-torch


[nltk_data] Downloading package punkt to /Users/jeanbapt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
# 2. Load CamemBERT and tokenizer
from transformers import CamembertTokenizer, CamembertModel
import torch

# Check for Metal support
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

tokenizer = CamembertTokenizer.from_pretrained("camembert/camembert-base")
model = CamembertModel.from_pretrained("camembert/camembert-base")
model = model.to(device)
model.eval()

Using device: mps


CamembertModel(
  (embeddings): CamembertEmbeddings(
    (word_embeddings): Embedding(32005, 768, padding_idx=0)
    (position_embeddings): Embedding(514, 768, padding_idx=0)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): CamembertEncoder(
    (layer): ModuleList(
      (0-11): 12 x CamembertLayer(
        (attention): CamembertAttention(
          (self): CamembertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): CamembertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
 

In [48]:
# 3. Sample French documents
documents = [
    # Climate and Environment
    "Le changement climatique menace la biodiversité mondiale et accélère l'extinction des espèces.",
    "Les océans jouent un rôle crucial dans la régulation du climat et l'absorption du CO2.",
    "La fonte des glaciers arctiques modifie les courants océaniques et affecte le climat global.",
    "Les forêts tropicales sont essentielles pour maintenir l'équilibre climatique de la planète.",
    
    # Renewable Energy
    "Les énergies renouvelables sont essentielles pour lutter contre le réchauffement climatique.",
    "L'énergie solaire devient de plus en plus accessible et efficace pour les particuliers.",
    "Les éoliennes offshore représentent une source importante d'énergie propre en Europe.",
    "La transition énergétique nécessite des investissements massifs en infrastructure.",
    
    # Technology and AI
    "L'intelligence artificielle transforme radicalement les méthodes de travail traditionnelles.",
    "Le machine learning révolutionne la recherche médicale et le diagnostic des maladies.",
    "Les algorithmes de deep learning permettent une meilleure compréhension du langage naturel.",
    "La robotique collaborative améliore la sécurité et l'efficacité dans l'industrie.",
    
    # French Culture
    "La gastronomie française est inscrite au patrimoine mondial de l'UNESCO.",
    "Le cinéma français est reconnu pour sa créativité et sa diversité artistique.",
    "Les traditions viticoles françaises se transmettent de génération en génération.",
    "Le patrimoine architectural français témoigne de siècles d'histoire et d'innovation.",
    
    # Science and Research
    "Les chercheurs français développent de nouveaux traitements contre le cancer.",
    "Les découvertes en physique quantique ouvrent de nouvelles perspectives technologiques.",
    "La recherche en biotechnologie permet des avancées majeures en médecine personnalisée.",
    "Les études sur le génome humain révèlent de nouveaux mécanismes biologiques.",
    
    # Biodiversity
    "La protection des abeilles est cruciale pour la préservation de la biodiversité.",
    "Les récifs coralliens abritent 25% de la vie marine mondiale.",
    "La déforestation menace des milliers d'espèces animales et végétales.",
    "Les zones humides sont des écosystèmes essentiels pour la biodiversité.",
    
    # Sustainable Development
    "L'agriculture biologique contribue à la préservation des sols et de la biodiversité.",
    "L'économie circulaire propose un modèle plus durable de consommation.",
    "Les villes intelligentes optimisent leur consommation d'énergie et réduisent leur impact environnemental.",
    "Le développement durable concilie progrès économique et protection de l'environnement.",
    
    # Education and Innovation
    "L'éducation numérique transforme les méthodes d'apprentissage traditionnelles.",
    "Les fab labs encouragent l'innovation et la créativité technologique.",
    "La formation continue devient essentielle dans un monde en constante évolution.",
    "Les nouvelles technologies facilitent l'accès à l'éducation pour tous."
]

In [49]:
# 4. ColBERT-style encoder
def encode(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.squeeze(0), inputs["attention_mask"].squeeze(0)
# Encode all documents
colbert_index = [(doc, *encode(doc)) for doc in documents]

In [50]:
# 5. BM25 Retriever
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

tokenized_corpus = [word_tokenize(doc.lower()) for doc in documents]
bm25 = BM25Okapi(tokenized_corpus)

In [51]:
# 6. Hybrid Retriever
import torch.nn.functional as F

def colbert_score(query, k=3):
    q_embed, q_mask = encode(query)
    scores = []
    for doc, d_embed, d_mask in colbert_index:
        sim = torch.einsum('id,jd->ij', q_embed, d_embed)
        maxsim = sim.max(dim=1).values
        score = maxsim.mean().item()
        scores.append((doc, score))
    return sorted(scores, key=lambda x: -x[1])[:k]

def bm25_score(query, k=3):
    tokenized_query = word_tokenize(query.lower())
    scores = bm25.get_scores(tokenized_query)
    ranked = sorted(enumerate(scores), key=lambda x: -x[1])[:k]
    return [(documents[i], scores[i]) for i, _ in ranked]

def hybrid_score(query, alpha=0.5, k=3):
    colbert_results = dict(colbert_score(query, k=10))
    bm25_results = dict(bm25_score(query, k=10))
    combined = {}
    for doc in set(colbert_results) | set(bm25_results):
        c = colbert_results.get(doc, 0)
        b = bm25_results.get(doc, 0)
        combined[doc] = alpha * c + (1 - alpha) * b
    return sorted(combined.items(), key=lambda x: -x[1])[:k]

In [52]:
# 7. Try a French query
query = "Quel est l'impact du climat sur la nature ?"
for doc, score in hybrid_score(query, alpha=0.8):
    print(f"{score:.4f} - {doc}")

17.7039 - Les océans jouent un rôle crucial dans la régulation du climat et l'absorption du CO2.
17.4164 - La recherche en biotechnologie permet des avancées majeures en médecine personnalisée.
16.2568 - Les forêts tropicales sont essentielles pour maintenir l'équilibre climatique de la planète.


# 8. DSPy Integration with Mistral via Ollama

In [53]:
# Wrap Mistral (Ollama) as DSPy LM
import dspy
import ollama
import time

class MistralOllamaLM(dspy.LM):
    def __init__(self, max_retries=3, timeout=30):
        super().__init__(model='mistral')
        self.max_retries = max_retries
        self.timeout = timeout
        self.client = ollama.Client()

    def __call__(self, prompt, **kwargs):
        for attempt in range(self.max_retries):
            try:
                response = self.client.chat(
                    model='mistral',
                    messages=[{"role": "user", "content": prompt}],
                    options={"timeout": self.timeout * 1000}
                )
                return response['message']['content']
            except Exception as e:
                if attempt == self.max_retries - 1:
                    raise Exception(f"Failed to get response from Ollama after {self.max_retries} attempts: {str(e)}")
                print(f"Attempt {attempt + 1} failed, retrying...")
                time.sleep(1)

lm = MistralOllamaLM()
dspy.settings.configure(lm=lm)

In [54]:
# DSPy Retriever wrapper using hybrid_score
from dspy.retrieve import Retrieve

class HybridDSPyRetriever(Retrieve):
    def __init__(self, alpha=0.8, k=3):
        super().__init__()
        self.alpha = alpha
        self.k = k

    def retrieve(self, query, k=None):
        results = hybrid_score(query, alpha=self.alpha, k=k or self.k)
        return [dspy.Passage(text=doc, score=score) for doc, score in results]

retriever = HybridDSPyRetriever()

In [56]:
# DSPy Pipeline Implementation
from dspy import Module, InputField, OutputField, Signature, Example

class HybridRetriever(Module):
    def __init__(self, hybrid_score_function, alpha=0.6, k=3):
        super().__init__()
        self.hybrid_score_function = hybrid_score_function
        self.alpha = alpha
        self.k = k

    def forward(self, query):
        results = self.hybrid_score_function(query, alpha=self.alpha, k=self.k)
        return results

class FrenchQAPipeline(Module):
    def __init__(self, retriever, alpha=0.6, k=3):
        super().__init__()
        self.retriever = retriever
        self.alpha = alpha
        self.k = k
        
    def forward(self, question):
        # Retrieve relevant passages
        # Use the retriever's hybrid_score_function correctly
        passages = hybrid_score(question, alpha=self.alpha, k=self.k)
        
        # Format context from passages
        context = "\n".join(f"- {doc}" for doc, _ in passages)
        
        # Create prompt for Mistral with improved instructions
        prompt = f"""En utilisant uniquement les informations fournies ci-dessous, réponds à la question en français.
        Si tu ne peux pas répondre avec ces informations, dis-le honnêtement.

        Contexte:
        {context}

        Question: {question}

        Réponse:"""
        
        # Get answer from Mistral
        response = lm(prompt)
        return response, passages

# Initialize the pipeline
retriever = HybridRetriever(hybrid_score_function=hybrid_score)
qa_pipeline = FrenchQAPipeline(retriever, alpha=0.7)

# Test queries with different focuses
test_queries = [
    "Quel est l'impact du changement climatique sur les océans ?",
    "Comment l'intelligence artificielle transforme-t-elle la médecine ?",
    "Quelles sont les traditions culturelles importantes en France ?",
    "Comment protéger la biodiversité et les écosystèmes ?",
    "Quel est le rôle des énergies renouvelables dans la transition énergétique ?",
]

print("Testing hybrid retrieval (alpha=0.7):\n")
for query in test_queries:
    print(f"\nQuestion: {query}")
    answer, sources = qa_pipeline(query)
    print(f"\nRéponse: {answer}")
    print("\nSources utilisées:")
    for doc, score in sources:
        print(f"{score:.4f} - {doc}")
    print("-" * 80)

Testing hybrid retrieval (alpha=0.7):


Question: Quel est l'impact du changement climatique sur les océans ?

Réponse:  Le changement climatique a un impact néfaste sur les océans. Il cause une augmentation des températures, qui a pour effet de faire fondre plus rapidement les glaciers et les icebergs, ce qui fait monter le niveau des océans. De plus, les océans absorbent du CO2, mais l'augmentation de la quantité de gaz à effet de serre qu'ils prennent peut entraîner une acidification des eaux, ce qui menace la vie marine et peut finalement avoir des conséquences sur notre propre écosystème.

Sources utilisées:
15.9521 - Les océans jouent un rôle crucial dans la régulation du climat et l'absorption du CO2.
14.3561 - Les forêts tropicales sont essentielles pour maintenir l'équilibre climatique de la planète.
13.6654 - La recherche en biotechnologie permet des avancées majeures en médecine personnalisée.
--------------------------------------------------------------------------------

